In [5]:
from bs4 import BeautifulSoup
import requests
import openpyxl
import re
from openpyxl import load_workbook
title=""
suffix=""
wb = load_workbook('GW.xlsx')
GCSsh=wb["GCS"]


In [6]:
def getinformation(row):
#function to accumulate datas from <div class="row"> input
    personprofile=row.find("div", {"class":"col-sm-6"})
    nameposdeplong=[personprofile.strip() for personprofile in personprofile.text.splitlines()]
    nameposdep=list(filter(None,nameposdeplong))
    first1=[]
    last1=[]
    name=nameposdep[0].split(" ")
    for index, namez in enumerate(name):
        if index==0:
            first1.append(namez)
        else:
            last1.append(namez)

    first=" ".join(first1)
    last=" ".join(last1)
    position=nameposdep[1]
    department=nameposdep[2]
    
    contact=row.find("div", {"class":"col-sm-6 text-right"})
    emailtelelong=[contact.strip() for contact in contact.text.splitlines()]
    emailtele=list(filter(None,emailtelelong))
    email=emailtele[0]
    if len(emailtele)>1 and emailtele[1].find("Tel:") != -1:
        telephone1=emailtele[1]
        telephone=telephone1.strip("Tel: ")
    else:
        telephone=None
    datas=[title, first, last, suffix, position, department, email, telephone]
    return list(filter (None, datas))



In [7]:
organisations1=[]
departmentnopage=[]
departmentnopageformat=[]

#creating a list of governmental departments
url="https://gcs.civilservice.gov.uk/membership/search/results?organisation=department-for-exiting-the-european-union&sub_organisation=&searchtype=department"
page=requests.get(url).content
soup = BeautifulSoup(page, 'html.parser')

for dep in soup.find("select").find_all("option"):
    value=dep.get("value")
    organisations1.append(value)
organisations2=list(filter(None,organisations1))

#creating list with search results for each department
for agency in organisations2:
    deps="https://gcs.civilservice.gov.uk/membership/search/results?organisation={}&sub_organisation=&searchtype=department&page=".format(agency)
    departmentnopage.append(deps)
    
for pages in departmentnopage:
    pages=pages+"{}"
    
    for i in range(1,23):
        gcspages=pages.format(i)
        page=requests.get(gcspages).content
        souper = BeautifulSoup(page, 'html.parser')
        
        if souper.find(text="No records found!"):
            continue
        else: 
            departmentnopageformat.append(souper)

In [8]:
#extracting rows from pages

for depnopag in departmentnopageformat:#depnopag=1page
    allrows=depnopag.find_all('div', {'class':'row'})#returns list with all rows on one page
    readyrows=[]
    rowstobegot1=[]
    
    for people in allrows:
        if people.find('a', {'class':'person-profile'}):
            rowstobegot1.append(people)
        else:
            continue

    readyrows=rowstobegot1[1:]
    for final in readyrows:
        a=getinformation(final)
        GCSsh.append(a)
wb.save('GW.xlsx')